In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample

# Veri setini yükleme
file_path = 'C:/Users/Mehmet Oruç Morgül/Downloads/UKR_refugee_cleaned.csv'
df_cleaned = pd.read_csv(file_path)

# LabelEncoder kullanarak object sütunlarını sayısal hale getirme
label_encoder = LabelEncoder()
for column in ['id', 'country', 'date', 'source']:
    df_cleaned[column] = label_encoder.fit_transform(df_cleaned[column])

# Hedef değişkeni ve özellikleri ayırma
X = df_cleaned.drop("individuals", axis=1)
y = df_cleaned["individuals"]

# Veri setini X ve y olarak birleştirme
df = pd.concat([X, y], axis=1)

# Çoğunluk ve azınlık sınıflarını ayırma
majority_class = df[df['individuals'] == df['individuals'].mode()[0]]
minority_classes = df[df['individuals'] != df['individuals'].mode()[0]]

# Azınlık sınıflarını çoğaltma
minority_upsampled = resample(minority_classes, 
                              replace=True,    # örneklerin yerine konarak seçilmesi
                              n_samples=len(majority_class),  # çoğunluk sınıfı ile aynı sayıda örnek
                              random_state=42) # rastgele tohum

# Veri setini yeniden birleştirme
df_balanced = pd.concat([majority_class, minority_upsampled])

# Hedef değişkeni ve özellikleri ayırma
X_balanced = df_balanced.drop("individuals", axis=1)
y_balanced = df_balanced["individuals"]

# Özellikleri standardize etme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_balanced)

# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_balanced, test_size=0.3, random_state=42)

# Modeli oluşturma ve eğitme
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
clf.fit(X_train, y_train)

# Test seti üzerinde modelin performansı
test_score = clf.score(X_test, y_test)

print("RandomForestClassifier score:", test_score)


RandomForestClassifier score: 0.5
